# Request Financial Statement Datasets

In [1]:
import bql
import pandas as pd
import utils.data_request_helper as helper
from utils.s3_helper import S3Helper

import os
import importlib
import json

from ipywidgets import IntProgress
from IPython.display import display

In [31]:
importlib.reload(helper)

<module 'utils.data_request_helper' from '/project/utils/data_request_helper.py'>

In [ ]:
# Index to use for point in time firms
index = 'INDU Index'
filename = 'data_quarterly_pit_indu_refresh_blended.json'
reporting_period = 'Q'

# rebalance dates for the index
rebalance_dates = ['2024-12-31',
        '2024-09-30',
        '2024-06-30',
        '2024-03-31',
        '2023-12-31',
        '2023-09-30',
        '2023-06-30',
        '2023-03-31',
        '2022-12-31',
        '2022-09-30',
        '2022-06-30',
        '2022-03-31',
        '2021-12-31',
        '2021-09-30',
        '2021-06-30',
        '2021-03-31',
        '2020-12-31',
        '2020-09-30',
        '2020-06-30',
        '2020-03-31',
        '2019-12-31',
        '2019-09-30',
        '2019-06-30',
        '2019-03-31',]

In [4]:
data_helper = helper.FinancialDataRequester(index_id=index,
                                            dataset_name='quarterly_pit_indu_blended',
                                            rebalance_dates=rebalance_dates,
                                            reporting_frequency=reporting_period)

In [5]:
df_rebalance_dates = data_helper.get_rebalance_dates()

100%|██████████| 24/24 [00:22<00:00,  1.04it/s]


In [6]:
all_data = data_helper.request_financial_data(df_rebalance_dates)

100%|█████████▉| 434/435 [27:23<00:03,  3.59s/it]

In [13]:
# Write the data to local storage
local_file = '/tmp/dow_quarterly_ltm.json'
with open(local_file, 'w') as f:
    json.dump(all_data, f)

In [9]:
s3_helper = S3Helper('tmp/fs')

In [14]:
s3_helper.add_file(local_filename=local_file)

### Request Annual datasets

In [32]:
# Index to use for point in time firms
index = 'INDU Index'
filename = 'data_annual_pit_dow.json'
reporting_period = 'A'

# rebalance dates for the index
rebalance_dates = ['2024-12-31',
        '2024-09-30',
        '2024-06-30',
        '2024-03-31',
        '2023-12-31',
        '2023-09-30',
        '2023-06-30',
        '2023-03-31',
        '2022-12-31',
        '2022-09-30',
        '2022-06-30',
        '2022-03-31',
        '2021-12-31',
        '2021-09-30',
        '2021-06-30',
        '2021-03-31',
        '2020-12-31',
        '2020-09-30',
        '2020-06-30',
        '2020-03-31',
        '2019-12-31',
        '2019-09-30',
        '2019-06-30',
        '2019-03-31',]

In [33]:
data_helper = helper.FinancialDataRequester(index_id=index,
                                            dataset_name='annual_pit_indu_blended',
                                            rebalance_dates=rebalance_dates,
                                            reporting_frequency=reporting_period)

In [35]:
all_data = data_helper.create_financial_dataset()

 99%|█████████▉| 145/146 [09:05<00:03,  3.76s/it]


ResponseError: Error encountered while processing universe list expression. Input failed to evaluate: "LIST([])".

In [5]:
from company_data import SecurityData

In [7]:
sec_helper = SecurityData('tmp/fs', 'dow_quarterly_ltm.json')

In [8]:
all_data = sec_helper.get_all_data()

In [27]:
meta = data_helper.create_financial_dataset(all_data)

In [30]:
meta['2020-04-23']['INTC UW Equity']['mt']

{'name': 'Intel Corp', 'figi': 'BBG000C0G1D1', 'sector': 'Technology'}

In [15]:
all_data['2020-04-23']['INTC UW Equity'].keys()

dict_keys(['is', 'bs', 'px', 'mt'])

In [17]:
df_rebalance_dates = data_helper.get_rebalance_dates()

100%|██████████| 24/24 [00:20<00:00,  1.19it/s]


In [31]:
all_data = data_helper.request_financial_data(df_rebalance_dates)

  0%|          | 0/146 [00:00<?, ?it/s]

                  ID_DATE      sector    ID_DATE          figi    ID_DATE  \
ID                                                                          
MSFT UQ Equity 2020-07-22  Technology 2020-07-22  BBG000BPH459 2020-07-22   

                          name  
ID                              
MSFT UQ Equity  Microsoft Corp  
MSFT UQ Equity


ValueError: Series index must be unique for orient='index'

In [8]:
all_data.keys()

dict_keys([])

In [20]:
df_rebalance_dates

PERIOD_END_DATE
AS_OF_DATE ID                            
2020-04-23 NKE UN Equity       2019-05-31
           HON UW Equity       2019-12-31
           MRK UN Equity       2019-12-31
           GS UN Equity        2019-12-31
           BA UN Equity        2019-12-31
...                                   ...
2025-02-25 IBM UN Equity       2024-12-31
2025-02-26 GS UN Equity        2024-12-31
           NVDA UQ Equity      2025-01-26
2025-03-05 CRM UN Equity       2025-01-31
2025-03-20 HD UN Equity        2025-02-02

[270 rows x 1 columns]